In [ ]:
def Untargeted_Attack_FGSM(img,model,EPOCH,PerRge):
    orig_label=np.argmax(model(img).data.cpu().numpy())
    print("orig_label={}:{}".format(orig_label,class_label[orig_label]))
    originallabel=Variable(torch.Tensor([float(orig_label)]).to(device).long())

    img.requires_grad = True
    for param in model.parameters():
        param.requires_grad = False

    theta=0.05
    min_=-3 
    max_= 3  
   
    img_o=img    
    optimizer = torch.optim.Adam([img])
    loss_func = torch.nn.CrossEntropyLoss()
    
    for epoch in range(EPOCH):
        output = model(img)   
        loss = loss_func(output, originallabel)            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        label=np.argmax(output.data.cpu().numpy())
            
        if epoch%10==0:
            print(f"Epoch {epoch}")
            print("label:{}->{} loss={:.6f}\n-------------------------------".format(orig_label,label,loss))  


        if label != orig_label:
            print(f"Epoch {epoch}")
            print("label:{}->{} loss={:.6f}\n-------------------------------".format(orig_label,label,loss)) 
            print("Attack successfully!")
            break
        

        img.data=img.data+torch.sign(img.grad.data)*theta 
        
        idex_1=np.where(abs(img.data.cpu())>=max_)
        if (idex_1 is not None and len(idex_1[0]) !=0):
            for j in range(len(idex_1[0])):
                print("idx_1={} over {}".format((idex_1[0][j],idex_1[1][j],idex_1[2][j],idex_1[3][j]),img.data[idex_1[0][j],idex_1[1][j],idex_1[2][j],idex_1[3][j]]))
                img.data[idex_1[0][j],idex_1[1][j],idex_1[2][j],idex_1[3][j]]=np.clip(img.data[idex_1[0][j],idex_1[1][j],idex_1[2][j],idex_1[3][j]].cpu(), min_, max_)

            
        img_diff=img-img_o  
        idex_2=np.where(abs(img_diff.data.cpu())>PerRge)
        if (idex_2 is not None and len(idex_2[0]) !=0):     
            for j in range(len(idex_2[0])):
                print("idx_2={} over {}".format((idex_2[0][j],idex_2[1][j],idex_2[2][j],idex_2[3][j]),img_diff.data[idex_2[0][j],idex_2[1][j],idex_2[2][j],idex_2[3][j]]))
                img_diff.data[idex_2[0][j],idex_2[1][j],idex_2[2][j],idex_2[3][j]]=np.clip(img_diff.data[idex_2[0][j],idex_2[1][j],idex_2[2][j],idex_2[3][j]].cpu(),-PerRge, PerRge)

        img=img_diff+img_o
        img = Variable(img.to(device).float())
        img.requires_grad = True
            
    preds_=Func.softmax(model(img),dim=1)
    probs_,idx_=preds_.data.squeeze().sort(0,True)
    probs_=probs_.data.cpu().numpy()
    idx_=idx_.data.cpu().numpy()
    for k_ in range(10):
        print('{}, {},  {:.5f}, {}'.format(k_+1, idx_[k_], probs_[k_], class_label[idx_[k_]]))
    
    adv= img.data.cpu().numpy()[0]
    adv = adv.transpose(1, 2, 0)
    

    adv = (adv * std) + mean
    adv = adv * 256.0
    adv = np.clip(adv, 0, 255).astype(np.uint8)     
    
   
    
    print("original {}:{}".format(orig_label,class_label[orig_label]))
    print("current {}:{}".format(label,class_label[label]))
    
          
    return orig_label,adv,label,epoch+1